In [51]:
import numpy as np

# Function created to initialize the Item vector - implement one that gets the data from a .txt file
# https://awari.com.br/vetor-python-aprenda-a-utilizar-a-linguagem-de-programacao-para-manipular-vetores/
def askForItens():
    qntd_itens = int(input("Type the number of items:"))
    #Create maze with dimension 2xItems    
    matriz = np.zeros((2, qntd_itens))
    
    ## Fill Valor
    for i in range(qntd_itens):
        matriz[0][i] = int(input(f"Type the Value for each item {i+1}:"))
    ## Fill Peso

    for i in range(qntd_itens):
        matriz[1, i] = float(input(f"Type the Weight for each item {i+1}: "))

    return matriz



In [52]:
import random
import math

# Calcular o valor total a mochila com os itens dentro dela
def calculate_value(solution, original, capacity):
    total_weight = 0
    total_value = 0

    # Loop para percorrer o vetor inteiro
    for i in range(len(solution)):
        # Calcular o peso e valor total para cada item que estiver presente 
        if(solution[i] == 1):
            # Coluna o item que foi selecionado
            total_weight += original[0][i]
            total_value += original[1][i]
            # Desconsidera se tiver ultrapassado a capacidade
            if(total_weight > capacity):
                # Retorna um valor negativo para indicar que estourou
                return -1
    return total_value

# Fornecer uma opção randomica de itens que estariam inseridos nessa primeira lista de solução
def initRandom(size):
    # 100% random! 
    randomVector = np.random.choice([True, False], size=size)
    return randomVector

In [ ]:
# Perguntar a capacidade total da Mochila
# capacity = int(input("Bag Capacity: "))
capacity = 9

# Comecar perguntando sobre os items
#originalValues = askForItens()
originalValues = np.array([[2, 3, 4, 5],[3, 4, 5, 6]])

# Definir estados Iniciais - Aleatório
initialSolution = initRandom(originalValues.shape[1])

print(f"Vetor Inicial: {initialSolution}")
currentValue = calculate_value(initialSolution, originalValues, capacity)
print(f"Valor Atual: {currentValue}")

# A melhor solução é a que se tem no momento
bestSolution = initialSolution
bestValue = currentValue

# Loop infinito - O critério de parada é se não vai aceitar mais variações ("Temperatura")
solutionFound = 0
while (solutionFound == 0):
    # Temperatura recebe um valor "aleatório" que vai dimiuindo conforme aumentar o número e iterações

    # Se a Temperatura for igual a Zero - acaba o algoritmo

    # Se T!= 0 : Escolhe vizinho aleatório

    # Se Valor VIZINHO for MAIOR que o valor ATUAL 
    # Próximo - Atual
    
    # Se for MAIOR: Atual = Próximo 
        
    # Se NÃO for MAIOR
        # Se existe a probabilidade de aceitar
            # Então Atual = Próximo
    solutionFound = 1